# **Hit 2**

**Versión de GPU**

In [ ]:
%%writefile hola_mundo.cu
#include <cstdio>
#include <cuda_runtime.h>

__global__ void hello_kernel(){

    int thread_in_block = threadIdx.x;
    int block_index = blockIdx.x;
    int threads_per_block = blockDim.x;
    int global_id = block_index * threads_per_block + thread_in_block;

    printf("Hola mundo desde el hilo global %d (block %d, thread %d)\n", global_id, block_index, thread_in_block);

}

int main(){
    //2 bloques de 4 hilos cada uno
    const int blocks = 2;
    const int threads_per_block = 4;

    hello_kernel<<<blocks, threads_per_block>>>();

    //Verificar errores
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        fprintf(stderr, "Error en el lanzamiento del kernel: %s\n", cudaGetErrorString(err));
        return 1;
    }

    //Verificar errores cuando la GPU termine
    err = cudaDeviceSynchronize();
    if (err != cudaSuccess) {
        fprintf(stderr, "Error despues de sincronizar: %s\n", cudaGetErrorString(err));
        return 1;
    }

    return 0;

}

Writing hola_mundo.cu


In [ ]:
!nvcc hola_mundo.cu -o hola_mundo -arch=sm_75

In [ ]:
!./hola_mundo

Hola mundo desde el hilo global 0 (block 0, thread 0)
Hola mundo desde el hilo global 1 (block 0, thread 1)
Hola mundo desde el hilo global 2 (block 0, thread 2)
Hola mundo desde el hilo global 3 (block 0, thread 3)
Hola mundo desde el hilo global 4 (block 1, thread 0)
Hola mundo desde el hilo global 5 (block 1, thread 1)
Hola mundo desde el hilo global 6 (block 1, thread 2)
Hola mundo desde el hilo global 7 (block 1, thread 3)


**Versión de CPU**

In [ ]:
blocks = 2
threads_per_block = 4

print("Version CPU del hola mundo")
for block in range(blocks):
    for thread in range(threads_per_block):
        global_id = block * threads_per_block + thread
        print(f"Hola mundo desde el hilo global {global_id} (block {block}, thread {thread})")

Version CPU del hola mundo
Hola mundo desde el hilo global 0 (block 0, thread 0)
Hola mundo desde el hilo global 1 (block 0, thread 1)
Hola mundo desde el hilo global 2 (block 0, thread 2)
Hola mundo desde el hilo global 3 (block 0, thread 3)
Hola mundo desde el hilo global 4 (block 1, thread 0)
Hola mundo desde el hilo global 5 (block 1, thread 1)
Hola mundo desde el hilo global 6 (block 1, thread 2)
Hola mundo desde el hilo global 7 (block 1, thread 3)


# **Hit 3**

In [ ]:
%%writefile ejemplo.cu

#include <iostream>
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/generate.h>
#include <thrust/sort.h>
#include <thrust/copy.h>
#include <thrust/random.h>

int main() {
    const int N = 1 << 20; //1M para prueba; cambiar a 32<<20 para algo mas realista pero costoso
    const int M = 8;

    thrust::default_random_engine rng(1337);
    thrust::uniform_int_distribution<int> dist;
    thrust::host_vector<int> h_vec(N);
    thrust::generate(h_vec.begin(), h_vec.end(), [&] { return dist(rng); });

    std::cout << "Antes (primeros " << M << "): ";
    for (int i = 0; i < M; ++i) std::cout << h_vec[i] << " ";
    std::cout << "\n";

    thrust::device_vector<int> d_vec = h_vec;
    thrust::sort(d_vec.begin(), d_vec.end());
    thrust::copy(d_vec.begin(), d_vec.begin() + M, h_vec.begin());

    std::cout << "Despues (primeros " << M << "): ";
    for (int i = 0; i < M; ++i) std::cout << h_vec[i] << " ";
    std::cout << "\n";

    return 0;
}

Writing ejemplo.cu


In [ ]:
!nvcc ejemplo.cu -o ejemplo -arch=sm_75 -O2

In [ ]:
!./ejemplo

Antes (primeros 8): 64538326 1478294467 18110393 180984444 336668598 1321185480 1122440121 294666980 
Despues (primeros 8): 1486 1554 1617 3454 5015 5354 6282 8234 


# **Hit 4**

In [ ]:
%%writefile md5_gpu.cu

#include <cstdio>
#include <cstdlib>
#include <cstdint>
#include <cstring>
#include <cuda_runtime.h>

__constant__ uint32_t dev_shifts[16] = { 7,12,17,22, 5,9,14,20, 4,11,16,23, 6,10,15,21 };
__constant__ uint32_t dev_sines[64] = {
    0xd76aa478,0xe8c7b756,0x242070db,0xc1bdceee,0xf57c0faf,0x4787c62a,0xa8304613,0xfd469501,
    0x698098d8,0x8b44f7af,0xffff5bb1,0x895cd7be,0x6b901122,0xfd987193,0xa679438e,0x49b40821,
    0xf61e2562,0xc040b340,0x265e5a51,0xe9b6c7aa,0xd62f105d,0x02441453,0xd8a1e681,0xe7d3fbc8,
    0x21e1cde6,0xc33707d6,0xf4d50d87,0x455a14ed,0xa9e3e905,0xfcefa3f8,0x676f02d9,0x8d2a4c8a,
    0xfffa3942,0x8771f681,0x6d9d6122,0xfde5380c,0xa4beea44,0x4bdecfa9,0xf6bb4b60,0xbebfbc70,
    0x289b7ec6,0xeaa127fa,0xd4ef3085,0x04881d05,0xd9d4d039,0xe6db99e5,0x1fa27cf8,0xc4ac5665,
    0xf4292244,0x432aff97,0xab9423a7,0xfc93a039,0x655b59c3,0x8f0ccc92,0xffeff47d,0x85845dd1,
    0x6fa87e4f,0xfe2ce6e0,0xa3014314,0x4e0811a1,0xf7537e82,0xbd3af235,0x2ad7d2bb,0xeb86d391
};

// rotación izquierda (device)
__device__ __forceinline__ uint32_t dev_left_rotate(uint32_t x, uint32_t c) {
    return (x << c) | (x >> (32 - c));
}

// transform que procesa un bloque de 64 bytes
__device__ void md5_transform_device(const uint8_t* chunk, uint32_t* h) {

    uint32_t M[16];
    for (int i = 0; i < 16; ++i) {
        M[i] = (uint32_t)chunk[i*4]
             | ((uint32_t)chunk[i*4+1] << 8)
             | ((uint32_t)chunk[i*4+2] << 16)
             | ((uint32_t)chunk[i*4+3] << 24);
    }

    uint32_t a = h[0], b = h[1], c = h[2], d = h[3];

    for (int i = 0; i < 64; ++i) {
        uint32_t F, g;
        if (i < 16) {
            F = (b & c) | ((~b) & d);
            g = i;
        } else if (i < 32) {
            F = (d & b) | ((~d) & c);
            g = (5*i + 1) & 15;
        } else if (i < 48) {
            F = b ^ c ^ d;
            g = (3*i + 5) & 15;
        } else {
            F = c ^ (b | (~d));
            g = (7*i) & 15;
        }

        uint32_t tmp = a + F + dev_sines[i] + M[g];
        a = d; d = c; c = b;

        uint32_t sh;
        if (i < 16) sh = dev_shifts[i % 4];
        else if (i < 32) sh = dev_shifts[4 + (i % 4)];
        else if (i < 48) sh = dev_shifts[8 + (i % 4)];
        else sh = dev_shifts[12 + (i % 4)];

        b = b + dev_left_rotate(tmp, sh);
    }

    h[0] += a; h[1] += b; h[2] += c; h[3] += d;
}

// kernel que procesa todo el mensaje padded (un hilo)
__global__ void md5_kernel(const uint8_t* d_msg, size_t d_len, uint32_t* d_out_h) {
    if (threadIdx.x != 0 || blockIdx.x != 0) return; // un solo hilo

    uint32_t h[4];
    h[0] = 0x67452301;
    h[1] = 0xefcdab89;
    h[2] = 0x98badcfe;
    h[3] = 0x10325476;

    size_t nblocks = d_len / 64;
    for (size_t i = 0; i < nblocks; ++i) {
        md5_transform_device(d_msg + i*64, h);
    }

    // escribir resultado en device memory
    d_out_h[0] = h[0];
    d_out_h[1] = h[1];
    d_out_h[2] = h[2];
    d_out_h[3] = h[3];
}

// ---------------- Host helpers (padding) ----------------
unsigned char* md5_pad_message_host(const unsigned char* initial_msg, size_t initial_len, size_t* out_len) {
    size_t new_len = initial_len + 1;
    while (new_len % 64 != 56) new_len++;
    new_len += 8;
    unsigned char* msg = (unsigned char*)malloc(new_len);
    if (!msg) return NULL;
    memcpy(msg, initial_msg, initial_len);
    msg[initial_len] = 0x80;
    memset(msg + initial_len + 1, 0, new_len - initial_len - 1);
    uint64_t bits_len = (uint64_t)initial_len * 8;
    for (int i = 0; i < 8; ++i) msg[new_len - 8 + i] = (unsigned char)((bits_len >> (8 * i)) & 0xFF);
    *out_len = new_len;
    return msg;
}

// ---------------- Main (host) ----------------
int main(int argc, char* argv[]) {
    if (argc < 2) {
        printf("Uso: %s \"texto a hashear\"\n", argv[0]);
        return 1;
    }
    const char* input = argv[1];
    size_t input_len = strlen(input);

    // padding en host
    size_t padded_len;
    unsigned char* padded = md5_pad_message_host((const unsigned char*)input, input_len, &padded_len);
    if (!padded) {
        fprintf(stderr, "Fallo malloc padding\n");
        return 1;
    }

    // reservar memoria device
    uint8_t* d_msg = nullptr;
    uint32_t* d_out = nullptr;
    cudaError_t err = cudaMalloc((void**)&d_msg, padded_len);
    if (err != cudaSuccess) { fprintf(stderr, "cudaMalloc msg: %s\n", cudaGetErrorString(err)); free(padded); return 1; }
    err = cudaMemcpy(d_msg, padded, padded_len, cudaMemcpyHostToDevice);
    if (err != cudaSuccess) { fprintf(stderr, "cudaMemcpy msg: %s\n", cudaGetErrorString(err)); cudaFree(d_msg); free(padded); return 1; }

    err = cudaMalloc((void**)&d_out, 4 * sizeof(uint32_t));
    if (err != cudaSuccess) { fprintf(stderr, "cudaMalloc out: %s\n", cudaGetErrorString(err)); cudaFree(d_msg); free(padded); return 1; }

    // lanzar kernel (1 bloque, 1 hilo)
    md5_kernel<<<1,1>>>(d_msg, padded_len, d_out);
    err = cudaGetLastError();
    if (err != cudaSuccess) { fprintf(stderr, "Launch error: %s\n", cudaGetErrorString(err)); cudaFree(d_msg); cudaFree(d_out); free(padded); return 1; }

    err = cudaDeviceSynchronize();
    if (err != cudaSuccess) { fprintf(stderr, "Sync error: %s\n", cudaGetErrorString(err)); cudaFree(d_msg); cudaFree(d_out); free(padded); return 1; }

    uint32_t h_out[4];
    err = cudaMemcpy(h_out, d_out, 4 * sizeof(uint32_t), cudaMemcpyDeviceToHost);
    if (err != cudaSuccess) { fprintf(stderr, "Memcpy out: %s\n", cudaGetErrorString(err)); cudaFree(d_msg); cudaFree(d_out); free(padded); return 1; }

    // imprimir en little-endian como MD5
    for (int i = 0; i < 4; ++i) {
        uint32_t v = h_out[i];
        printf("%02x%02x%02x%02x", v & 0xFF, (v >> 8) & 0xFF, (v >> 16) & 0xFF, (v >> 24) & 0xFF);
    }
    printf("\n");

    cudaFree(d_msg);
    cudaFree(d_out);
    free(padded);
    return 0;
}

Writing md5_gpu.cu


In [ ]:
!nvcc md5_gpu.cu -o md5_gpu -arch=sm_75 -O2

In [ ]:
!./md5_gpu "hola mundo"

0ad066a5d29f3f2a2a1c7c17dd082a79


**Versión de CPU**

In [ ]:
%%writefile md5_cpu.py
import sys
import hashlib

def md5_of_string(s: str) -> str:
    """Devuelve el MD5 (hex) de la cadena s."""
    # hashlib.md5 necesita bytes, por eso encode()
    m = hashlib.md5(s.encode('utf-8'))
    return m.hexdigest()

def main():
    if len(sys.argv) < 2:
        print("Uso: python md5_cpu.py \"texto a hashear\"")
        sys.exit(1)

    texto = sys.argv[1]
    resultado = md5_of_string(texto)
    print(resultado)

if __name__ == "__main__":
    main()

#python md5_cpu.py "hola mundo"

Writing md5_cpu.py


In [ ]:
!python md5_cpu.py "hola mundo"

0ad066a5d29f3f2a2a1c7c17dd082a79


# **Hit 5**

In [22]:
%%writefile minero_cuda.cu
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <stdint.h>
#include <cuda_runtime.h>

constexpr int RANGE_FROM = 0;
constexpr int RANGE_TO   = 850000;

// ---------------- MD5 ----------------
__constant__ uint32_t dev_shifts[16] = { 7,12,17,22, 5,9,14,20, 4,11,16,23, 6,10,15,21 };
__constant__ uint32_t dev_sines[64] = {
    0xd76aa478,0xe8c7b756,0x242070db,0xc1bdceee,0xf57c0faf,0x4787c62a,0xa8304613,0xfd469501,
    0x698098d8,0x8b44f7af,0xffff5bb1,0x895cd7be,0x6b901122,0xfd987193,0xa679438e,0x49b40821,
    0xf61e2562,0xc040b340,0x265e5a51,0xe9b6c7aa,0xd62f105d,0x02441453,0xd8a1e681,0xe7d3fbc8,
    0x21e1cde6,0xc33707d6,0xf4d50d87,0x455a14ed,0xa9e3e905,0xfcefa3f8,0x676f02d9,0x8d2a4c8a,
    0xfffa3942,0x8771f681,0x6d9d6122,0xfde5380c,0xa4beea44,0x4bdecfa9,0xf6bb4b60,0xbebfbc70,
    0x289b7ec6,0xeaa127fa,0xd4ef3085,0x04881d05,0xd9d4d039,0xe6db99e5,0x1fa27cf8,0xc4ac5665,
    0xf4292244,0x432aff97,0xab9423a7,0xfc93a039,0x655b59c3,0x8f0ccc92,0xffeff47d,0x85845dd1,
    0x6fa87e4f,0xfe2ce6e0,0xa3014314,0x4e0811a1,0xf7537e82,0xbd3af235,0x2ad7d2bb,0xeb86d391
};

// rotación izquierda
__device__ __forceinline__ uint32_t dev_left_rotate(uint32_t x, uint32_t c) {
    return (x << c) | (x >> (32 - c));
}

// transform que procesa un bloque de 64 bytes
__device__ void md5_transform_device(const uint8_t* chunk, uint32_t* h) {
    uint32_t M[16];
    for (int i = 0; i < 16; ++i) {
        M[i] = (uint32_t)chunk[i*4]
             | ((uint32_t)chunk[i*4+1] << 8)
             | ((uint32_t)chunk[i*4+2] << 16)
             | ((uint32_t)chunk[i*4+3] << 24);
    }

    uint32_t a = h[0], b = h[1], c = h[2], d = h[3];

    for (int i = 0; i < 64; ++i) {
        uint32_t F, g;
        if (i < 16) {
            F = (b & c) | ((~b) & d);
            g = i;
        } else if (i < 32) {
            F = (d & b) | ((~d) & c);
            g = (5*i + 1) & 15;
        } else if (i < 48) {
            F = b ^ c ^ d;
            g = (3*i + 5) & 15;
        } else {
            F = c ^ (b | (~d));
            g = (7*i) & 15;
        }

        uint32_t tmp = a + F + dev_sines[i] + M[g];
        a = d; d = c; c = b;

        uint32_t sh;
        if (i < 16) sh = dev_shifts[i % 4];
        else if (i < 32) sh = dev_shifts[4 + (i % 4)];
        else if (i < 48) sh = dev_shifts[8 + (i % 4)];
        else sh = dev_shifts[12 + (i % 4)];

        b = b + dev_left_rotate(tmp, sh);
    }

    h[0] += a; h[1] += b; h[2] += c; h[3] += d;
}


__device__ void cuda_md5_singleblock(const uint8_t* initial_msg, int initial_len, uint8_t* digest_out) {
    // estados iniciales
    uint32_t h[4];
    h[0] = 0x67452301;
    h[1] = 0xefcdab89;
    h[2] = 0x98badcfe;
    h[3] = 0x10325476;

    // preparar bloque en stack
    uint8_t chunk[64];
    // inicializar con ceros
    for (int i = 0; i < 64; ++i) chunk[i] = 0;

    // copiar mensaje
    for (int i = 0; i < initial_len; ++i) chunk[i] = initial_msg[i];

    // padding
    chunk[initial_len] = 0x80;

    // longitud en bits, little-endian
    uint64_t bits_len = (uint64_t)initial_len * 8ULL;
    for (int i = 0; i < 8; ++i) chunk[56 + i] = (uint8_t)((bits_len >> (8 * i)) & 0xFF);

    // procesar un bloque
    md5_transform_device(chunk, h);

    // escribir digest_out en formato little-endian
    for (int i = 0; i < 4; ++i) {
        uint32_t v = h[i];
        digest_out[i*4 + 0] = (uint8_t)(v & 0xFF);
        digest_out[i*4 + 1] = (uint8_t)((v >> 8) & 0xFF);
        digest_out[i*4 + 2] = (uint8_t)((v >> 16) & 0xFF);
        digest_out[i*4 + 3] = (uint8_t)((v >> 24) & 0xFF);
    }
}

// -------------------- Device helpers --------------------
__device__ void int_to_str_device(int num, char *str) {
    int i = 0;
    if (num == 0) { str[0] = '0'; str[1] = '\0'; return; }
    unsigned int n = (num < 0) ? (unsigned int)(-num) : (unsigned int)num;
    char tmp[32];
    while (n > 0) { tmp[i++] = '0' + (n % 10); n /= 10; }
    int j = 0;
    if (num < 0) str[j++] = '-';
    while (i > 0) str[j++] = tmp[--i];
    str[j] = '\0';
}

__device__ int d_strlen(const char* s) {
    int i = 0; while (s[i] != '\0') ++i; return i;
}

__device__ bool starts_with_device(const char* hash, const char* prefix, int prefix_len) {
    for (int i = 0; i < prefix_len; ++i) if (hash[i] != prefix[i]) return false;
    return true;
}

__device__ void bytes_to_hex_device(const uint8_t* bytes, char* hex_out) {
    const char digits[] = "0123456789abcdef";
    for (int i = 0; i < 16; ++i) {
        hex_out[i * 2]     = digits[(bytes[i] >> 4) & 0x0F];
        hex_out[i * 2 + 1] = digits[bytes[i] & 0x0F];
    }
    hex_out[32] = '\0';
}

// -------------------- Kernel con conteo de intentos --------------------
__global__ void mine_kernel_with_attempts(
    const char* input, int input_len,
    const char* prefix, int prefix_len,
    int from, int to,
    int* found_flag, int* out_nonce, char* out_hash,
    unsigned long long* attempts_global
) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;

    const int BUFFER_MAX = 2048;
    char nonce_str[32];
    char buffer[BUFFER_MAX];
    uint8_t md5_raw[16];
    char md5_hex[33];

    unsigned long long local_count = 0;
    const unsigned long long FLUSH_EVERY = 1024ULL;

    for (int nonce = from + idx; nonce <= to; nonce += stride) {
        if (*found_flag) {
            if (local_count) atomicAdd(attempts_global, local_count);
            return;
        }

        int_to_str_device(nonce, nonce_str);
        int nonce_len = d_strlen(nonce_str);

        // construir buffer: nonce + input
        if (nonce_len + input_len >= BUFFER_MAX) {
            // evitar overflow
            continue;
        }
        // copiar nonce
        for (int i = 0; i < nonce_len; ++i) buffer[i] = nonce_str[i];
        // copiar input despues
        for (int i = 0; i < input_len; ++i) buffer[nonce_len + i] = input[i];
        int total_len = nonce_len + input_len;

        // single-block requiere total_len <= 55
        if (total_len > 55) {
            // contar intento tambien
            local_count++;
            if (local_count >= FLUSH_EVERY) { atomicAdd(attempts_global, local_count); local_count = 0; }
            continue;
        }

        // calcular MD5
        cuda_md5_singleblock((const uint8_t*)buffer, total_len, md5_raw);
        bytes_to_hex_device(md5_raw, md5_hex);

        local_count++;
        if (local_count >= FLUSH_EVERY) {
            atomicAdd(attempts_global, local_count);
            local_count = 0;
        }

        if (starts_with_device(md5_hex, prefix, prefix_len)) {
            int prev = atomicCAS(found_flag, 0, 1);
            if (local_count) atomicAdd(attempts_global, local_count);
            if (prev == 0) {
                *out_nonce = nonce;
                for (int i = 0; i < 33; ++i) out_hash[i] = md5_hex[i];
            }
            return;
        }
    }

    if (local_count) atomicAdd(attempts_global, local_count);
    return;
}

// -------------------- Host main --------------------
int main(int argc, char *argv[]) {
    if (argc != 4) {
        fprintf(stderr, "Uso: %s <prefix> <input> <output>\n", argv[0]);
        fprintf(stderr, "RANGO hardcodeado en el codigo: [%d .. %d]\n", RANGE_FROM, RANGE_TO);
        return 1;
    }

    const char* prefix = argv[1];
    const char* input = argv[2];
    const char* output = argv[3];

    int input_len = (int)strlen(input);
    int prefix_len = (int)strlen(prefix);

    char *d_input = NULL, *d_prefix = NULL, *d_hash = NULL;
    int *d_found = NULL, *d_nonce = NULL;
    unsigned long long *d_attempts = NULL;

    cudaMalloc(&d_input, (input_len + 1) * sizeof(char));
    cudaMalloc(&d_prefix, (prefix_len + 1) * sizeof(char));
    cudaMalloc(&d_hash, 33 * sizeof(char));
    cudaMalloc(&d_found, sizeof(int));
    cudaMalloc(&d_nonce, sizeof(int));
    cudaMalloc(&d_attempts, sizeof(unsigned long long));

    cudaMemcpy(d_input, input, (input_len + 1) * sizeof(char), cudaMemcpyHostToDevice);
    cudaMemcpy(d_prefix, prefix, (prefix_len + 1) * sizeof(char), cudaMemcpyHostToDevice);
    cudaMemset(d_found, 0, sizeof(int));
    cudaMemset(d_attempts, 0, sizeof(unsigned long long));

    // parametros launching
    int threads = 256;
    int blocks = 1024; // aprox 262k hilos

    mine_kernel_with_attempts<<<blocks, threads>>>(
        d_input, input_len,
        d_prefix, prefix_len,
        RANGE_FROM, RANGE_TO,
        d_found, d_nonce, d_hash,
        d_attempts
    );

    cudaDeviceSynchronize();

    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        fprintf(stderr, "CUDA kernel error: %s\n", cudaGetErrorString(err));
        // cleanup
        if (d_input) cudaFree(d_input);
        if (d_prefix) cudaFree(d_prefix);
        if (d_hash) cudaFree(d_hash);
        if (d_found) cudaFree(d_found);
        if (d_nonce) cudaFree(d_nonce);
        if (d_attempts) cudaFree(d_attempts);
        return 1;
    }

    int found = 0;
    int nonce = 0;
    char hash_host[33]; memset(hash_host, 0, sizeof(hash_host));
    unsigned long long attempts_host = 0ULL;

    cudaMemcpy(&found, d_found, sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(&attempts_host, d_attempts, sizeof(unsigned long long), cudaMemcpyDeviceToHost);

    if (found) {
        cudaMemcpy(&nonce, d_nonce, sizeof(int), cudaMemcpyDeviceToHost);
        cudaMemcpy(hash_host, d_hash, 33 * sizeof(char), cudaMemcpyDeviceToHost);
    }

    // salida: "<nonce> <hash> <attempts>"
    FILE* f = fopen(output, "w");
    if (!f) {
        fprintf(stderr, "No se pudo abrir archivo %s\n", output);
    } else {
        if (found) {
            fprintf(f, "%d %s %llu", nonce, hash_host, attempts_host);
        } else {
            fprintf(f, "0 ");
            for (int i = 0; i < 32; ++i) fputc('0', f);
            fprintf(f, " %llu", attempts_host);
        }
        fclose(f);
    }

    if (d_input) cudaFree(d_input);
    if (d_prefix) cudaFree(d_prefix);
    if (d_hash) cudaFree(d_hash);
    if (d_found) cudaFree(d_found);
    if (d_nonce) cudaFree(d_nonce);
    if (d_attempts) cudaFree(d_attempts);

    return 0;
}

Overwriting minero_cuda.cu


In [23]:
!nvcc -arch=sm_75 -O2 minero_cuda.cu -o minero_cuda

In [24]:
%%writefile minero_gpu.py
import subprocess
import sys
from pathlib import Path
import uuid

BASE_DIR = Path(".").resolve()

if sys.platform.startswith("win"):
    CUDA_BIN = BASE_DIR / "minero_cuda.exe"
else:
    CUDA_BIN = BASE_DIR / "minero_cuda"

if not CUDA_BIN.exists():
    raise RuntimeError(f"No se encontro el binario CUDA: {CUDA_BIN}")

def ejecutar_minero(prefix, input_val):

    output_file = BASE_DIR / f"gpu_output_{uuid.uuid4().hex}.txt"

    cmd = [
        str(CUDA_BIN),
        prefix,
        input_val,
        str(output_file),
    ]

    proc = subprocess.run(cmd, capture_output=True, text=True)

    if proc.returncode != 0:
        return {"numero": 0, "hash_md5_result": "", "intentos": 0}

    contenido = output_file.read_text().strip()

    try:
        numero, hash_md5, intentos = contenido.split()
        resultado = {
            "numero": int(numero),
            "hash_md5_result": hash_md5 if int(numero) > 0 else "",
            "intentos": int(intentos),
        }
    except Exception:
        resultado = {"numero": 0, "hash_md5_result": "", "intentos": 0}

    try:
        output_file.unlink()
    except Exception:
        pass

    return resultado

Overwriting minero_gpu.py


In [41]:
from minero_gpu import ejecutar_minero

resultado = ejecutar_minero("00", "hola mundo")
print(resultado)

{'numero': 21469, 'hash_md5_result': '00633e36003628dfa029e841be3f6218', 'intentos': 41088}


**Versión CPU**

In [42]:
%%writefile minero_cpu.py
import hashlib
RANGE_START = 0
RANGE_END   = 850000

def ejecutar_minero_cpu(prefix, input_val):

    intentos = 0

    for nonce in range(RANGE_START, RANGE_END + 1):
        mensaje = f"{nonce}{input_val}".encode("utf-8")
        hash_md5 = hashlib.md5(mensaje).hexdigest()
        intentos += 1

        if hash_md5.startswith(prefix):
            return {
                "numero": nonce,
                "hash_md5_result": hash_md5,
                "intentos": intentos,
            }

    # Si no se encuentra nada
    return {
        "numero": 0,
        "hash_md5_result": "",
        "intentos": intentos,
    }

if __name__ == "__main__":
    res = ejecutar_minero_cpu("00", "hola mundo")
    print(res)


Overwriting minero_cpu.py


In [44]:
from minero_cpu import ejecutar_minero_cpu

resultado_cpu = ejecutar_minero_cpu("00", "hola mundo")
print(resultado_cpu)

{'numero': 181, 'hash_md5_result': '00618bcc021f5a393c2b348f3c85318c', 'intentos': 182}


# **Hit 6**

In [ ]:
import time
from statistics import mean
from minero_gpu import ejecutar_minero

# --- CONFIG ---
cadena = "hola mundo"
prefijos = ["0","00","000","0000","00000","000000","0000000"]  # prefijos a probar
REPETICIONES = 1   # cuantas ejecuciones por cada prefijo
# ----------------

resumen = []
all_runs = []

for pref in prefijos:
    print(f"\n--- Prefijo: '{pref}' ---")
    tiempos = []
    hashes_por_s_list = []
    resultados_rep = []

    for rep in range(1, REPETICIONES + 1):
        print(f"Ejecutando rep {rep}/{REPETICIONES} ...", end="", flush=True)
        t0 = time.perf_counter()
        res = ejecutar_minero(pref, cadena)
        t1 = time.perf_counter()
        tiempo = t1 - t0

        numero = int(res.get("numero", 0))
        hash_md5 = res.get("hash_md5_result", "")
        intentos = int(res.get("intentos", 0))

        hashes_por_s = round(intentos/tiempo, 1) if tiempo > 0 else None

        fila = {
            "prefijo": pref,
            "repeticion": rep,
            "nonce": numero,
            "hash": hash_md5,
            "intentos": intentos,
            "tiempo_s": round(tiempo, 6),
            "hashes_por_s": hashes_por_s,
        }

        resultados_rep.append(fila)
        all_runs.append(fila)
        tiempos.append(tiempo)
        if hashes_por_s is not None:
            hashes_por_s_list.append(hashes_por_s)

        print(f"nonce={numero} intentos={intentos} tiempo={fila['tiempo_s']}s hashes/s={fila['hashes_por_s']}")

    resumen_pref = {
        "prefijo": pref,
        "repeticiones": REPETICIONES,
        "encontrado_en_alguna": any(r["nonce"] != 0 for r in resultados_rep),
        "nonce_ejemplo": next((r["nonce"] for r in resultados_rep if r["nonce"] != 0), 0),
        "hash_ejemplo": next((r["hash"] for r in resultados_rep if r["nonce"] != 0), ""),
        "intentos_promedio": int(mean(r["intentos"] for r in resultados_rep)),
        "tiempo_min_s": round(min(tiempos), 6),
        "tiempo_prom_s": round(mean(tiempos), 6),
        "tiempo_max_s": round(max(tiempos), 6),
        "hashes_s_prom": round(mean(hashes_por_s_list), 1) if hashes_por_s_list else None
    }
    resumen.append(resumen_pref)

# imprimir resumen por prefijo
print("\n\n--- RESUMEN POR PREFIJO ---")
print("pref | found | nonce | intentos_prom | tiempo_prom_s | hashes/s_prom")
for r in resumen:
    print(f"{r['prefijo']:5} | {str(r['encontrado_en_alguna']):5} | {r['nonce_ejemplo']:9} | {r['intentos_promedio']:13} | {r['tiempo_prom_s']:12} | {r['hashes_s_prom']}")

# determinar el prefijo más largo encontrado
prefijos_encontrados = [r for r in resumen if r["encontrado_en_alguna"]]
if prefijos_encontrados:
    mejor = max(prefijos_encontrados, key=lambda x: len(x["prefijo"]))
    print("\nPrefijo más largo encontrado:", repr(mejor["prefijo"]))
    print("Nonce:", mejor["nonce_ejemplo"])
    print("Hash:", mejor["hash_ejemplo"])
    print("Tiempo promedio (s):", mejor["tiempo_prom_s"])
else:
    print("\nNo se encontró ninguna coincidencia para los prefijos probados en los rangos internos del binario CUDA.")


--- Prefijo: '0' ---
Ejecutando rep 1/1 ... OK -> nonce=7172 intentos=40960 tiempo=0.402066s hashes/s=101873.8

--- Prefijo: '00' ---
Ejecutando rep 1/1 ... OK -> nonce=16715 intentos=40960 tiempo=0.279414s hashes/s=146592.4

--- Prefijo: '000' ---
Ejecutando rep 1/1 ... OK -> nonce=7590 intentos=40960 tiempo=0.248233s hashes/s=165006.5

--- Prefijo: '0000' ---
Ejecutando rep 1/1 ... OK -> nonce=27522 intentos=40960 tiempo=0.235293s hashes/s=174080.7

--- Prefijo: '00000' ---
Ejecutando rep 1/1 ... OK -> nonce=0 intentos=200001 tiempo=0.242415s hashes/s=825035.2

--- Prefijo: '000000' ---
Ejecutando rep 1/1 ... OK -> nonce=0 intentos=200001 tiempo=0.226611s hashes/s=882574.4

--- Prefijo: '0000000' ---
Ejecutando rep 1/1 ... OK -> nonce=0 intentos=200001 tiempo=0.227455s hashes/s=879299.4


--- RESUMEN POR PREFIJO ---
pref | found | nonce_ej | intentos_prom | tiempo_prom_s | hashes/s_prom
0     | True  |      7172 |         40960 |     0.402066 | 101873.8
00    | True  |     16715 |  

**Versión CPU**

In [ ]:
import time
from statistics import mean
from minero_cpu import ejecutar_minero_cpu

# --- CONFIG ---
cadena = "hola mundo"
# prefijos a probar
prefijos = ["0","00","000","0000","00000","000000","0000000"]
REPETICIONES = 1
# ----------------

def medir_prefijo(pref, repeticiones):
    detalles = []
    for rep in range(1, repeticiones + 1):
        t0 = time.perf_counter()
        res = ejecutar_minero_cpu(pref, cadena)
        t1 = time.perf_counter()
        tiempo = t1 - t0

        numero = int(res.get("numero", 0))
        hash_md5 = res.get("hash_md5_result", "")
        intentos = int(res.get("intentos", 0))
        hashes_por_s = round(intentos / tiempo, 1) if tiempo > 0 else None

        detalles.append({
            "prefijo": pref,
            "repeticion": rep,
            "nonce": numero,
            "hash": hash_md5,
            "intentos": intentos,
            "tiempo_s": round(tiempo, 3),
            "hashes_por_s": hashes_por_s,
        })

        print(f"pref='{pref}' rep={rep} -> nonce={numero} intentos={intentos} tiempo={round(tiempo,3)}s hashes/s={hashes_por_s}")
    return detalles

def main():

    resumen = []

    for pref in prefijos:
        detalles = medir_prefijo(pref, REPETICIONES)

        tiempos = [d["tiempo_s"] for d in detalles]
        intentos_prom = int(mean(d["intentos"] for d in detalles))
        hashes_prom = round(mean(d["hashes_por_s"] for d in detalles if d["hashes_por_s"] is not None), 1) if any(d["hashes_por_s"] is not None for d in detalles) else None
        encontrado = any(d["nonce"] != 0 for d in detalles)
        nonce_ej = next((d["nonce"] for d in detalles if d["nonce"] != 0), 0)
        hash_ej = next((d["hash"] for d in detalles if d["hash"] != ""), "")

        resumen.append({
            "prefijo": pref,
            "nonce": nonce_ej,
            "encontrado": encontrado,
            "intentos_prom": intentos_prom,
            "tiempo_min": round(min(tiempos), 3),
            "tiempo_prom": round(mean(tiempos), 3),
            "tiempo_max": round(max(tiempos), 3),
            "hashes_s_prom": hashes_prom,
            "hash_ej": hash_ej,
        })

    # imprimir tabla resumen similar al informe
    print("\n--- RESUMEN POR PREFIJO ---\n")
    print("| Prefijo  | Nonce encontrado | Intentos | Tiempo (s) | Hashes/s (aprox.) | Hash (inicio) |")
    print("|----------|-----------------:|---------:|-----------:|------------------:|---------------:|")
    for r in resumen:
        nonce_str = f"{r['nonce']}" if r['encontrado'] else "–"
        hash_start = (r["hash_ej"][:8] + "...") if r["hash_ej"] else "—"
        hashes_str = f"{r['hashes_s_prom']}" if r['hashes_s_prom'] is not None else "—"
        print(f"| `{r['prefijo']}`{' '*(6-len(r['prefijo']))} | {nonce_str:16} | {r['intentos_prom']:8} | {r['tiempo_prom']:9} | {hashes_str:18} | {hash_start:13} |")

    # determinar prefijo más largo encontrado
    prefijos_encontrados = [r for r in resumen if r["encontrado"]]
    if prefijos_encontrados:
        mejor = max(prefijos_encontrados, key=lambda x: len(x["prefijo"]))
        print("\nPrefijo más largo encontrado:", repr(mejor["prefijo"]))
        print("Nonce:", mejor["nonce"])
        print("Hash:", mejor["hash_ej"][:32] if mejor["hash_ej"] else "")
        print("Tiempo promedio (s):", mejor["tiempo_prom"])
    else:
        print("\nNo se encontró ninguna coincidencia para los prefijos probados (en el rango 0..200000).")

if __name__ == "__main__":
    main()

pref='0' rep=1 -> nonce=4 intentos=5 tiempo=0.0s hashes/s=215684.6
pref='00' rep=1 -> nonce=181 intentos=182 tiempo=0.0s hashes/s=821507.2
pref='000' rep=1 -> nonce=5290 intentos=5291 tiempo=0.005s hashes/s=1069551.6
pref='0000' rep=1 -> nonce=27522 intentos=27523 tiempo=0.029s hashes/s=965074.8
pref='00000' rep=1 -> nonce=0 intentos=200001 tiempo=0.184s hashes/s=1088596.8
pref='000000' rep=1 -> nonce=0 intentos=200001 tiempo=0.186s hashes/s=1076854.3
pref='0000000' rep=1 -> nonce=0 intentos=200001 tiempo=0.182s hashes/s=1097013.2

--- RESUMEN POR PREFIJO ---

| Prefijo  | Nonce encontrado | Intentos | Tiempo (s) | Hashes/s (aprox.) | Hash (inicio) |
|----------|-----------------:|---------:|-----------:|------------------:|---------------:|
| `0`      | 4                |        5 |       0.0 | 215684.6           | 06b0f77e...   |
| `00`     | 181              |      182 |       0.0 | 821507.2           | 00618bcc...   |
| `000`    | 5290             |     5291 |     0.005 | 1069551.6

# **Hit 7**

In [ ]:
%%writefile minero_cuda_modificado.cu
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <stdint.h>
#include <cuda_runtime.h>

constexpr int RANGE_FROM = 0;
constexpr int RANGE_TO   = 200000;

// ---------------- MD5 ----------------
__constant__ uint32_t dev_shifts[16] = { 7,12,17,22, 5,9,14,20, 4,11,16,23, 6,10,15,21 };
__constant__ uint32_t dev_sines[64] = {
    0xd76aa478,0xe8c7b756,0x242070db,0xc1bdceee,0xf57c0faf,0x4787c62a,0xa8304613,0xfd469501,
    0x698098d8,0x8b44f7af,0xffff5bb1,0x895cd7be,0x6b901122,0xfd987193,0xa679438e,0x49b40821,
    0xf61e2562,0xc040b340,0x265e5a51,0xe9b6c7aa,0xd62f105d,0x02441453,0xd8a1e681,0xe7d3fbc8,
    0x21e1cde6,0xc33707d6,0xf4d50d87,0x455a14ed,0xa9e3e905,0xfcefa3f8,0x676f02d9,0x8d2a4c8a,
    0xfffa3942,0x8771f681,0x6d9d6122,0xfde5380c,0xa4beea44,0x4bdecfa9,0xf6bb4b60,0xbebfbc70,
    0x289b7ec6,0xeaa127fa,0xd4ef3085,0x04881d05,0xd9d4d039,0xe6db99e5,0x1fa27cf8,0xc4ac5665,
    0xf4292244,0x432aff97,0xab9423a7,0xfc93a039,0x655b59c3,0x8f0ccc92,0xffeff47d,0x85845dd1,
    0x6fa87e4f,0xfe2ce6e0,0xa3014314,0x4e0811a1,0xf7537e82,0xbd3af235,0x2ad7d2bb,0xeb86d391
};

// rotación izquierda
__device__ __forceinline__ uint32_t dev_left_rotate(uint32_t x, uint32_t c) {
    return (x << c) | (x >> (32 - c));
}

// transform que procesa un bloque de 64 bytes
__device__ void md5_transform_device(const uint8_t* chunk, uint32_t* h) {
    uint32_t M[16];
    for (int i = 0; i < 16; ++i) {
        M[i] = (uint32_t)chunk[i*4]
             | ((uint32_t)chunk[i*4+1] << 8)
             | ((uint32_t)chunk[i*4+2] << 16)
             | ((uint32_t)chunk[i*4+3] << 24);
    }

    uint32_t a = h[0], b = h[1], c = h[2], d = h[3];

    for (int i = 0; i < 64; ++i) {
        uint32_t F, g;
        if (i < 16) {
            F = (b & c) | ((~b) & d);
            g = i;
        } else if (i < 32) {
            F = (d & b) | ((~d) & c);
            g = (5*i + 1) & 15;
        } else if (i < 48) {
            F = b ^ c ^ d;
            g = (3*i + 5) & 15;
        } else {
            F = c ^ (b | (~d));
            g = (7*i) & 15;
        }

        uint32_t tmp = a + F + dev_sines[i] + M[g];
        a = d; d = c; c = b;

        uint32_t sh;
        if (i < 16) sh = dev_shifts[i % 4];
        else if (i < 32) sh = dev_shifts[4 + (i % 4)];
        else if (i < 48) sh = dev_shifts[8 + (i % 4)];
        else sh = dev_shifts[12 + (i % 4)];

        b = b + dev_left_rotate(tmp, sh);
    }

    h[0] += a; h[1] += b; h[2] += c; h[3] += d;
}


__device__ void cuda_md5_singleblock(const uint8_t* initial_msg, int initial_len, uint8_t* digest_out) {
    // estados iniciales
    uint32_t h[4];
    h[0] = 0x67452301;
    h[1] = 0xefcdab89;
    h[2] = 0x98badcfe;
    h[3] = 0x10325476;

    // preparar bloque en stack
    uint8_t chunk[64];
    // inicializar con ceros
    for (int i = 0; i < 64; ++i) chunk[i] = 0;

    // copiar mensaje
    for (int i = 0; i < initial_len; ++i) chunk[i] = initial_msg[i];

    // padding
    chunk[initial_len] = 0x80;

    // longitud en bits, little-endian
    uint64_t bits_len = (uint64_t)initial_len * 8ULL;
    for (int i = 0; i < 8; ++i) chunk[56 + i] = (uint8_t)((bits_len >> (8 * i)) & 0xFF);

    // procesar un bloque
    md5_transform_device(chunk, h);

    // escribir digest_out en formato little-endian
    for (int i = 0; i < 4; ++i) {
        uint32_t v = h[i];
        digest_out[i*4 + 0] = (uint8_t)(v & 0xFF);
        digest_out[i*4 + 1] = (uint8_t)((v >> 8) & 0xFF);
        digest_out[i*4 + 2] = (uint8_t)((v >> 16) & 0xFF);
        digest_out[i*4 + 3] = (uint8_t)((v >> 24) & 0xFF);
    }
}

// -------------------- Device helpers --------------------
__device__ void int_to_str_device(int num, char *str) {
    int i = 0;
    if (num == 0) { str[0] = '0'; str[1] = '\0'; return; }
    unsigned int n = (num < 0) ? (unsigned int)(-num) : (unsigned int)num;
    char tmp[32];
    while (n > 0) { tmp[i++] = '0' + (n % 10); n /= 10; }
    int j = 0;
    if (num < 0) str[j++] = '-';
    while (i > 0) str[j++] = tmp[--i];
    str[j] = '\0';
}

__device__ int d_strlen(const char* s) {
    int i = 0; while (s[i] != '\0') ++i; return i;
}

__device__ bool starts_with_device(const char* hash, const char* prefix, int prefix_len) {
    for (int i = 0; i < prefix_len; ++i) if (hash[i] != prefix[i]) return false;
    return true;
}

__device__ void bytes_to_hex_device(const uint8_t* bytes, char* hex_out) {
    const char digits[] = "0123456789abcdef";
    for (int i = 0; i < 16; ++i) {
        hex_out[i * 2]     = digits[(bytes[i] >> 4) & 0x0F];
        hex_out[i * 2 + 1] = digits[bytes[i] & 0x0F];
    }
    hex_out[32] = '\0';
}

// -------------------- Kernel con conteo de intentos --------------------
__global__ void mine_kernel_with_attempts(
    const char* input, int input_len,
    const char* prefix, int prefix_len,
    int from, int to,
    int* found_flag, int* out_nonce, char* out_hash,
    unsigned long long* attempts_global
) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;

    const int BUFFER_MAX = 2048;
    char nonce_str[32];
    char buffer[BUFFER_MAX];
    uint8_t md5_raw[16];
    char md5_hex[33];

    unsigned long long local_count = 0;
    const unsigned long long FLUSH_EVERY = 1024ULL;

    for (int nonce = from + idx; nonce <= to; nonce += stride) {
        if (*found_flag) {
            if (local_count) atomicAdd(attempts_global, local_count);
            return;
        }

        int_to_str_device(nonce, nonce_str);
        int nonce_len = d_strlen(nonce_str);

        // construir buffer: nonce + input
        if (nonce_len + input_len >= BUFFER_MAX) {
            // evitar overflow
            continue;
        }
        // copiar nonce
        for (int i = 0; i < nonce_len; ++i) buffer[i] = nonce_str[i];
        // copiar input despues
        for (int i = 0; i < input_len; ++i) buffer[nonce_len + i] = input[i];
        int total_len = nonce_len + input_len;

        // single-block requiere total_len <= 55
        if (total_len > 55) {
            // contar intento tambien
            local_count++;
            if (local_count >= FLUSH_EVERY) { atomicAdd(attempts_global, local_count); local_count = 0; }
            continue;
        }

        // calcular MD5
        cuda_md5_singleblock((const uint8_t*)buffer, total_len, md5_raw);
        bytes_to_hex_device(md5_raw, md5_hex);

        local_count++;
        if (local_count >= FLUSH_EVERY) {
            atomicAdd(attempts_global, local_count);
            local_count = 0;
        }

        if (starts_with_device(md5_hex, prefix, prefix_len)) {
            int prev = atomicCAS(found_flag, 0, 1);
            if (local_count) atomicAdd(attempts_global, local_count);
            if (prev == 0) {
                *out_nonce = nonce;
                for (int i = 0; i < 33; ++i) out_hash[i] = md5_hex[i];
            }
            return;
        }
    }

    if (local_count) atomicAdd(attempts_global, local_count);
    return;
}

// -------------------- Host main --------------------
int main(int argc, char *argv[]) {
    if (argc != 6) {
        fprintf(stderr, "Uso: %s <prefix> <input> <output> <from> <to>\n", argv[0]);
        fprintf(stderr, "Ejemplo: %s 00 \"hola mundo\" resultado.txt 0 200000\n", argv[0]);
        fprintf(stderr, "RANGO por defecto en el codigo: [%d .. %d]\n", RANGE_FROM, RANGE_TO);
        return 1;
    }

    const char* prefix = argv[1];
    const char* input = argv[2];
    const char* output = argv[3];
    int range_from = atoi(argv[4]);
    int range_to   = atoi(argv[5]);

    if (range_from > range_to) {
        // intercambiar si el usuario se equivocó
        int tmp = range_from; range_from = range_to; range_to = tmp;
    }

    int input_len = (int)strlen(input);
    int prefix_len = (int)strlen(prefix);

    char *d_input = NULL, *d_prefix = NULL, *d_hash = NULL;
    int *d_found = NULL, *d_nonce = NULL;
    unsigned long long *d_attempts = NULL;

    cudaMalloc(&d_input, (input_len + 1) * sizeof(char));
    cudaMalloc(&d_prefix, (prefix_len + 1) * sizeof(char));
    cudaMalloc(&d_hash, 33 * sizeof(char));
    cudaMalloc(&d_found, sizeof(int));
    cudaMalloc(&d_nonce, sizeof(int));
    cudaMalloc(&d_attempts, sizeof(unsigned long long));

    cudaMemcpy(d_input, input, (input_len + 1) * sizeof(char), cudaMemcpyHostToDevice);
    cudaMemcpy(d_prefix, prefix, (prefix_len + 1) * sizeof(char), cudaMemcpyHostToDevice);
    cudaMemset(d_found, 0, sizeof(int));
    cudaMemset(d_attempts, 0, sizeof(unsigned long long));

    // parametros launching
    int threads = 256;
    int blocks = 1024; // aprox 262k hilos

    mine_kernel_with_attempts<<<blocks, threads>>>(
        d_input, input_len,
        d_prefix, prefix_len,
        range_from, range_to,
        d_found, d_nonce, d_hash,
        d_attempts
    );

    cudaDeviceSynchronize();

    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        fprintf(stderr, "CUDA kernel error: %s\n", cudaGetErrorString(err));
        // cleanup
        if (d_input) cudaFree(d_input);
        if (d_prefix) cudaFree(d_prefix);
        if (d_hash) cudaFree(d_hash);
        if (d_found) cudaFree(d_found);
        if (d_nonce) cudaFree(d_nonce);
        if (d_attempts) cudaFree(d_attempts);
        return 1;
    }

    int found = 0;
    int nonce = 0;
    char hash_host[33]; memset(hash_host, 0, sizeof(hash_host));
    unsigned long long attempts_host = 0ULL;

    cudaMemcpy(&found, d_found, sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(&attempts_host, d_attempts, sizeof(unsigned long long), cudaMemcpyDeviceToHost);

    if (found) {
        cudaMemcpy(&nonce, d_nonce, sizeof(int), cudaMemcpyDeviceToHost);
        cudaMemcpy(hash_host, d_hash, 33 * sizeof(char), cudaMemcpyDeviceToHost);
    } else {
        printf("No se encontró ninguna solución en el rango [%d .. %d]\n", range_from, range_to);
    }

    // salida: "<nonce> <hash> <attempts>"
    FILE* f = fopen(output, "w");
    if (!f) {
        fprintf(stderr, "No se pudo abrir archivo %s\n", output);
    } else {
        if (found) {
            fprintf(f, "%d %s %llu", nonce, hash_host, attempts_host);
        } else {
            fprintf(f, "0 ");
            for (int i = 0; i < 32; ++i) fputc('0', f);
            fprintf(f, " %llu", attempts_host);
        }
        fclose(f);
    }

    if (d_input) cudaFree(d_input);
    if (d_prefix) cudaFree(d_prefix);
    if (d_hash) cudaFree(d_hash);
    if (d_found) cudaFree(d_found);
    if (d_nonce) cudaFree(d_nonce);
    if (d_attempts) cudaFree(d_attempts);

    return 0;
}

Overwriting minero_cuda_modificado.cu


In [ ]:
!nvcc -arch=sm_75 -O2 minero_cuda_modificado.cu -o minero_cuda_modificado

In [ ]:
%%writefile minero_gpu_mod.py
import subprocess
import sys
from pathlib import Path
import uuid

BASE_DIR = Path(".").resolve()

if sys.platform.startswith("win"):
    CUDA_BIN = BASE_DIR / "minero_cuda_modificado.exe"
else:
    CUDA_BIN = BASE_DIR / "minero_cuda_modificado"

if not CUDA_BIN.exists():
    raise RuntimeError(f"No se encontro el binario CUDA: {CUDA_BIN}")

def ejecutar_minero(prefix, input_val, from_=0, to_=200000):
    """
    Ejecuta el binario CUDA pasando el prefijo, la cadena y el rango [from_, to_].
    Retorna dict: {"numero": int, "hash_md5_result": str, "intentos": int}
    """
    output_file = BASE_DIR / f"gpu_output_{uuid.uuid4().hex}.txt"

    cmd = [
        str(CUDA_BIN),
        prefix,
        input_val,
        str(output_file),
        str(int(from_)),
        str(int(to_)),
    ]

    proc = subprocess.run(cmd, capture_output=True, text=True)

    if proc.returncode != 0:
        return {"numero": 0, "hash_md5_result": "", "intentos": 0}

    contenido = ""
    try:
        contenido = output_file.read_text().strip()
    except Exception:
        return {"numero": 0, "hash_md5_result": "", "intentos": 0}

    try:
        numero, hash_md5, intentos = contenido.split()
        resultado = {
            "numero": int(numero),
            "hash_md5_result": hash_md5 if int(numero) > 0 else "",
            "intentos": int(intentos),
        }
    except Exception:
        resultado = {"numero": 0, "hash_md5_result": "", "intentos": 0}

    try:
        output_file.unlink()
    except Exception:
        pass

    return resultado


if __name__ == "__main__":
    res = ejecutar_minero("00", "hola mundo", 0, 200000)
    print(res)

Writing minero_gpu_mod.py


In [ ]:
from minero_gpu_mod import ejecutar_minero

res = ejecutar_minero("00", "hola mundo", 0, 500000)
print(res)

{'numero': 16715, 'hash_md5_result': '005508cc14203dfd30819aaf0482fa35', 'intentos': 41216}


In [ ]:
%%writefile pruebas_hit_7.py
import sys
import time
from statistics import mean
from minero_gpu_mod import ejecutar_minero

if len(sys.argv) != 3:
    print("Ejemplo: python pruebas_hit_7.py 0 200000")
    sys.exit(1)

RANGO_FROM = int(sys.argv[1])
RANGO_TO   = int(sys.argv[2])

cadena = "hola mundo"
prefijos = ["0","00","000","0000","00000","000000","0000000"]
REPETICIONES = 1


def medir_prefijo(pref, repeticiones):
    detalles = []
    for rep in range(1, repeticiones + 1):
        t0 = time.perf_counter()
        res = ejecutar_minero(pref, cadena, RANGO_FROM, RANGO_TO)
        t1 = time.perf_counter()
        tiempo = t1 - t0

        numero = int(res.get("numero", 0))
        hash_md5 = res.get("hash_md5_result", "")
        intentos = int(res.get("intentos", 0))
        hashes_por_s = round(intentos / tiempo, 1) if tiempo > 0 else None

        detalles.append({
            "prefijo": pref,
            "repeticion": rep,
            "nonce": numero,
            "hash": hash_md5,
            "intentos": intentos,
            "tiempo_s": round(tiempo, 3),
            "hashes_por_s": hashes_por_s,
        })

        print(f"pref='{pref}' rep={rep} -> nonce={numero} intentos={intentos} tiempo={round(tiempo,3)}s hashes/s={hashes_por_s}")
    return detalles


def main():

    resumen = []

    for pref in prefijos:
        detalles = medir_prefijo(pref, REPETICIONES)

        tiempos = [d["tiempo_s"] for d in detalles]
        intentos_prom = int(mean(d["intentos"] for d in detalles))
        hashes_prom = round(mean(d["hashes_por_s"] for d in detalles if d["hashes_por_s"] is not None), 1) if any(d["hashes_por_s"] is not None for d in detalles) else None
        encontrado = any(d["nonce"] != 0 for d in detalles)
        nonce_ej = next((d["nonce"] for d in detalles if d["nonce"] != 0), 0)
        hash_ej = next((d["hash"] for d in detalles if d["hash"] != ""), "")

        resumen.append({
            "prefijo": pref,
            "nonce": nonce_ej,
            "encontrado": encontrado,
            "intentos_prom": intentos_prom,
            "tiempo_min": round(min(tiempos), 3),
            "tiempo_prom": round(mean(tiempos), 3),
            "tiempo_max": round(max(tiempos), 3),
            "hashes_s_prom": hashes_prom,
            "hash_ej": hash_ej,
        })

    print("\n--- RESUMEN POR PREFIJO ---\n")
    print("| Prefijo  | Nonce encontrado | Intentos | Tiempo (s) | Hashes/s (aprox.) | Hash (inicio) |")
    print("|----------|-----------------:|---------:|-----------:|------------------:|---------------:|")

    for r in resumen:
        nonce_str = f"{r['nonce']}" if r['encontrado'] else "–"
        hash_start = (r["hash_ej"][:8] + "...") if r["hash_ej"] else "—"
        hashes_str = f"{r['hashes_s_prom']}" if r['hashes_s_prom'] is not None else "—"

        print(f"| `{r['prefijo']}`{' '*(6-len(r['prefijo']))} | {nonce_str:16} | {r['intentos_prom']:8} | {r['tiempo_prom']:9} | {hashes_str:18} | {hash_start:13} |")

    prefijos_encontrados = [r for r in resumen if r["encontrado"]]
    if prefijos_encontrados:
        mejor = max(prefijos_encontrados, key=lambda x: len(x["prefijo"]))
        print("\nPrefijo más largo encontrado:", repr(mejor["prefijo"]))
        print("Nonce:", mejor["nonce"])
        print("Hash:", mejor["hash_ej"][:32] if mejor["hash_ej"] else "")
        print("Tiempo promedio (s):", mejor["tiempo_prom"])
    else:
        print(f"\nNo se encontró ninguna coincidencia en el rango {RANGO_FROM}..{RANGO_TO}.")


if __name__ == "__main__":
    main()

Overwriting pruebas_hit_7.py


In [ ]:
!python3 pruebas_hit_7.py 0 850000

pref='0' rep=1 -> nonce=12355 intentos=40992 tiempo=0.389s hashes/s=105340.3
pref='00' rep=1 -> nonce=21469 intentos=41216 tiempo=0.268s hashes/s=153866.5
pref='000' rep=1 -> nonce=12485 intentos=57280 tiempo=0.241s hashes/s=237881.5
pref='0000' rep=1 -> nonce=290797 intentos=100864 tiempo=0.239s hashes/s=421475.6
pref='00000' rep=1 -> nonce=677029 intentos=539249 tiempo=0.236s hashes/s=2286855.1
pref='000000' rep=1 -> nonce=0 intentos=850001 tiempo=0.26s hashes/s=3273937.9
pref='0000000' rep=1 -> nonce=0 intentos=850001 tiempo=0.26s hashes/s=3272273.3

--- RESUMEN POR PREFIJO ---

| Prefijo  | Nonce encontrado | Intentos | Tiempo (s) | Hashes/s (aprox.) | Hash (inicio) |
|----------|-----------------:|---------:|-----------:|------------------:|---------------:|
| `0`      | 12355            |    40992 |     0.389 | 105340.3           | 04a467f4...   |
| `00`     | 21469            |    41216 |     0.268 | 153866.5           | 00633e36...   |
| `000`    | 12485            |    57280 |

**Versión CPU**

In [ ]:
%%writefile minero_cpu_hit_7.py
import hashlib
import argparse

# valores por defecto
DEFAULT_RANGE_START = 0
DEFAULT_RANGE_END = 200000

def ejecutar_minero(prefix, input_val, range_start=DEFAULT_RANGE_START, range_end=DEFAULT_RANGE_END):

    intentos = 0

    for nonce in range(range_start, range_end + 1):
        mensaje = f"{nonce}{input_val}".encode("utf-8")
        hash_md5 = hashlib.md5(mensaje).hexdigest()
        intentos += 1

        if hash_md5.startswith(prefix):
            return {
                "numero": nonce,
                "hash_md5_result": hash_md5,
                "intentos": intentos,
            }

    # Si no se encuentra nada
    return {
        "numero": 0,
        "hash_md5_result": "",
        "intentos": intentos,
    }

def _crear_parser():
    p = argparse.ArgumentParser(description="Minero CPU por fuerza bruta (MD5).")
    p.add_argument("--prefix", "-p", default="00", help='Prefijo que debe cumplir el hash (ej: "00")')
    p.add_argument("--input", "-i", dest="input_val", default="hola mundo", help='Texto a concatenar con el nonce')
    p.add_argument("--start", "-s", type=int, default=DEFAULT_RANGE_START, help="Nonce inicial (inclusive)")
    p.add_argument("--end", "-e", type=int, default=DEFAULT_RANGE_END, help="Nonce final (inclusive)")
    return p

if __name__ == "__main__":
    parser = _crear_parser()
    args = parser.parse_args()

    resultado = ejecutar_minero_cpu(args.prefix, args.input_val, args.start, args.end)
    print(resultado)

Overwriting minero_cpu_hit_7.py


In [ ]:
from minero_cpu_hit_7 import ejecutar_minero_cpu

resultado_cpu = ejecutar_minero_cpu("00", "hola mundo", range_start=0, range_end=85000)
print(resultado_cpu)

{'numero': 181, 'hash_md5_result': '00618bcc021f5a393c2b348f3c85318c', 'intentos': 182}


In [ ]:
%%writefile pruebas_hit_7_cpu.py
import sys
import time
from statistics import mean
from minero_cpu_hit_7 import ejecutar_minero

if len(sys.argv) != 3:
    print("Ejemplo: python pruebas_hit_7_cpu.py 0 200000")
    sys.exit(1)

RANGO_FROM = int(sys.argv[1])
RANGO_TO   = int(sys.argv[2])

cadena = "hola mundo"
prefijos = ["0","00","000","0000","00000","000000","0000000"]
REPETICIONES = 1


def medir_prefijo(pref, repeticiones):
    detalles = []

    for rep in range(1, repeticiones + 1):
        t0 = time.perf_counter()
        res = ejecutar_minero(pref, cadena, RANGO_FROM, RANGO_TO)
        t1 = time.perf_counter()

        tiempo = t1 - t0
        numero = int(res.get("numero", 0))
        hash_md5 = res.get("hash_md5_result", "")
        intentos = int(res.get("intentos", 0))
        hashes_por_s = round(intentos / tiempo, 1) if tiempo > 0 else None

        detalles.append({
            "prefijo": pref,
            "repeticion": rep,
            "nonce": numero,
            "hash": hash_md5,
            "intentos": intentos,
            "tiempo_s": round(tiempo, 3),
            "hashes_por_s": hashes_por_s,
        })

        print(
            f"pref='{pref}' rep={rep} -> "
            f"nonce={numero} intentos={intentos} "
            f"tiempo={round(tiempo,3)}s hashes/s={hashes_por_s}"
        )

    return detalles


def main():
    resumen = []

    for pref in prefijos:
        detalles = medir_prefijo(pref, REPETICIONES)

        tiempos = [d["tiempo_s"] for d in detalles]
        intentos_prom = int(mean(d["intentos"] for d in detalles))
        hashes_prom = (
            round(mean(d["hashes_por_s"] for d in detalles if d["hashes_por_s"] is not None), 1)
            if any(d["hashes_por_s"] is not None for d in detalles)
            else None
        )

        encontrado = any(d["nonce"] != 0 for d in detalles)
        nonce_ej = next((d["nonce"] for d in detalles if d["nonce"] != 0), 0)
        hash_ej = next((d["hash"] for d in detalles if d["hash"] != ""), "")

        resumen.append({
            "prefijo": pref,
            "nonce": nonce_ej,
            "encontrado": encontrado,
            "intentos_prom": intentos_prom,
            "tiempo_min": round(min(tiempos), 3),
            "tiempo_prom": round(mean(tiempos), 3),
            "tiempo_max": round(max(tiempos), 3),
            "hashes_s_prom": hashes_prom,
            "hash_ej": hash_ej,
        })

    print("\n--- RESUMEN POR PREFIJO (CPU) ---\n")
    print("| Prefijo  | Nonce encontrado | Intentos | Tiempo (s) | Hashes/s (aprox.) | Hash (inicio) |")
    print("|----------|-----------------:|---------:|-----------:|------------------:|---------------:|")

    for r in resumen:
        nonce_str = f"{r['nonce']}" if r["encontrado"] else "–"
        hash_start = (r["hash_ej"][:8] + "...") if r["hash_ej"] else "—"
        hashes_str = f"{r['hashes_s_prom']}" if r["hashes_s_prom"] is not None else "—"

        print(
            f"| `{r['prefijo']}`{' '*(6-len(r['prefijo']))} | "
            f"{nonce_str:16} | {r['intentos_prom']:8} | "
            f"{r['tiempo_prom']:9} | {hashes_str:18} | {hash_start:13} |"
        )

    prefijos_encontrados = [r for r in resumen if r["encontrado"]]

    if prefijos_encontrados:
        mejor = max(prefijos_encontrados, key=lambda x: len(x["prefijo"]))
        print("\nPrefijo más largo encontrado:", repr(mejor["prefijo"]))
        print("Nonce:", mejor["nonce"])
        print("Hash:", mejor["hash_ej"][:32])
        print("Tiempo promedio (s):", mejor["tiempo_prom"])
    else:
        print(f"\nNo se encontró ninguna coincidencia en el rango {RANGO_FROM}..{RANGO_TO}.")


if __name__ == "__main__":
    main()

Overwriting pruebas_hit_7_cpu.py


In [ ]:
!python pruebas_hit_7_cpu.py 0 850000

pref='0' rep=1 -> nonce=4 intentos=5 tiempo=0.0s hashes/s=213995.3
pref='00' rep=1 -> nonce=181 intentos=182 tiempo=0.0s hashes/s=903620.4
pref='000' rep=1 -> nonce=5290 intentos=5291 tiempo=0.005s hashes/s=1099506.7
pref='0000' rep=1 -> nonce=27522 intentos=27523 tiempo=0.026s hashes/s=1067971.9
pref='00000' rep=1 -> nonce=677029 intentos=677030 tiempo=0.624s hashes/s=1085360.7
pref='000000' rep=1 -> nonce=0 intentos=850001 tiempo=0.789s hashes/s=1076732.7
pref='0000000' rep=1 -> nonce=0 intentos=850001 tiempo=1.236s hashes/s=687767.0

--- RESUMEN POR PREFIJO (CPU) ---

| Prefijo  | Nonce encontrado | Intentos | Tiempo (s) | Hashes/s (aprox.) | Hash (inicio) |
|----------|-----------------:|---------:|-----------:|------------------:|---------------:|
| `0`      | 4                |        5 |       0.0 | 213995.3           | 06b0f77e...   |
| `00`     | 181              |      182 |       0.0 | 903620.4           | 00618bcc...   |
| `000`    | 5290             |     5291 |     0.005 